In [ ]:
!pip install pandas scikit-learn numpy

In [ ]:
!pip install datasets

In [ ]:
pip install kagglehub

In [ ]:
import kagglehub
kagglehub.login()

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import pandas as pd
import kagglehub
dataset_path = kagglehub.dataset_download('manuelthimoty/eda-final')
df = pd.read_csv(f"{dataset_path}/eda_final.csv") 
print(df.head())


In [ ]:
#manuelthimoty_clean_data_daq_path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# PRE PROCESSING

In [ ]:
df=df.drop_duplicates(subset=['Deskripsi'],keep='first')

In [ ]:
df

In [ ]:
df=df[['lama hukuman (bulan)','Deskripsi']]

In [ ]:
import numpy as np
df['lama hukuman (bulan)'] = df['lama hukuman (bulan)'].astype(np.float32)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

Make datset from pandas of train and test

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
train_dataset

In [ ]:
test_dataset

Defining the tokenizer and tokenize the train and test dataset

In [ ]:
pip install transformers

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

def tokenize(examples):
    return tokenizer(examples["Deskripsi"], padding="max_length", truncation=True,max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize, batched=True)

In [ ]:
tokenized_train_dataset = tokenized_train_dataset.rename_column("lama hukuman (bulan)", "labels")
tokenized_test_dataset = tokenized_test_dataset.rename_column("lama hukuman (bulan)", "labels")

In [ ]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['Deskripsi', '__index_level_0__'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(['Deskripsi', '__index_level_0__'])

# Defining the model
In this case, we use indobert-base-p1, and since is regression, num_labels=1

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", num_labels=1)

We define the error compute metrics, we use RMSE

In [ ]:
import torch
device = torch.device("cuda")
torch.cuda.is_available()
model.to(device)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Untuk regresi, logits adalah output langsungnya
    predictions = logits[:, 0]
    rmse = np.sqrt(mean_squared_error(labels, predictions))
    return {"rmse": rmse}

We define the training argument

In [ ]:
pip install 'accelerate>=0.26.0'

In [ ]:
NUM_EPOCH=10

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./hasil_fine_tuning", # Folder untuk menyimpan hasil
    eval_strategy="epoch",      # Lakukan evaluasi setiap selesai satu epoch
    save_strategy="epoch",            # Simpan model setiap selesai satu epoch
    learning_rate=2e-5,               # ini umum buat bert
    per_device_train_batch_size=8,   # Ukuran batch untuk training
    per_device_eval_batch_size=8,    # Ukuran batch untuk evaluasi
    num_train_epochs=NUM_EPOCH,               # Jumlah epoch training (sesuain ae sih, makin banyak makin bagus, cuman pasti makin lama)
    weight_decay=0.01,                # Teknik regularisasi
    load_best_model_at_end=True,      # Muat model terbaik di akhir training
    metric_for_best_model="rmse",     # Gunakan RMSE sebagai patokan model terbaik
    greater_is_better=False           # Semakin kecil RMSE, semakin baik
)

We initiate the trainer

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# # Bekukan semua parameter model terlebih dahulu
# for param in model.parameters():
#     param.requires_grad = False

# # Buka bekuan (unfreeze) untuk layer-layer yang ingin dilatih.
# # Kita akan melatih 6 encoder layer teratas dan layer classifier.

# # Unfreeze the top 6 encoder layers (layer 6 to 11)
# for i in range(6, 12):
#     for param in model.bert.encoder.layer[i].parameters():
#         param.requires_grad = True

# # Unfreeze the pooler layer (bertanggung jawab untuk output [CLS] token)
# for param in model.bert.pooler.parameters():
#     param.requires_grad = True

# # Unfreeze the final classifier layer
# for param in model.classifier.parameters():
#     param.requires_grad = True

# # Verifikasi layer mana yang trainable
# print("--- Status Trainable Parameter ---")
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"✅ Trainable: {name}")

# TRAIN

In [ ]:
dataset

In [ ]:
tokenized_train_dataset

In [ ]:
tokenized_test_dataset

In [ ]:
trainer.train()

In [ ]:
#pip install --upgrade transformers accelerate

In [ ]:
# tokenized_train_dataset = tokenized_train_dataset.remove_columns(['Deskripsi', '__index_level_0__'])
# tokenized_test_dataset = tokenized_test_dataset.remove_columns(['Deskripsi', '__index_level_0__'])
